In [1]:
#!/usr/bin/python3

In [2]:
__author__="Emily Hua"

Now you have to create some text files that will allow Kaldi to communicate with your audio data. Consider these files as 'must be done'. Each file that you will create in this section (and in Language data section as well) can be considered as a text file with some number of strings (each string in a new line). These strings need to be sorted. If you will encounter any sorting issues you can use Kaldi scripts for checking (utils/validate_data_dir.sh) and fixing (utils/fix_data_dir.sh) data order. And for you information - utils directory will be attached to your project in Tools attachment section.

In [175]:
import os
import re
import sys
import glob
from shutil import rmtree
from collections import defaultdict 

In [8]:
parent_path = os.path.split(os.getcwd())[0]
print (parent_path)

/Volumes/STARTRACK/deep-learning/code-switch/speech-to-text/codeswitch


In [9]:
ls -l ../audio/train/NC03FBX

total 199680
-rw-r--r--  1 yehua  staff  51933995 Apr 13 17:59 NC03FBX_020101.flac
-rw-r--r--  1 yehua  staff  50297952 Apr 13 17:59 NC03FBX_020201.flac


In kaldi/egs/code-switch directory, create a folder **data**. Then create **test** and **train** subfolders inside. Create in each subfolder following files (so you have files named in the same way in test and train subfolders but they relate to two different data sets that you created before):

In [29]:
def makedir(dirt):
    r"""create data/train or data/test directory
    """
    directory = parent_path + "/data/" + dirt
    if not os.path.exists(directory):
        os.makedirs(directory)

In [30]:
makedir("train")
makedir("test")

In [17]:
ls -l ../data

total 0
drwxr-xr-x  3 yehua  staff  102 Apr  7 22:55 lang/
drwxr-xr-x  4 yehua  staff  136 Apr  7 22:54 local/
drwxr-xr-x  9 yehua  staff  306 Apr  4 17:09 test/
drwxr-xr-x  9 yehua  staff  306 Apr  4 17:27 train/


In [18]:
ls -l ../data/train

total 23448
-rw-r--r--  1 yehua  staff  2505625 Apr  6 21:30 segments
-rw-r--r--@ 1 yehua  staff     1600 Apr  6 21:30 spk2gender
-rw-r--r--  1 yehua  staff  3929042 Apr  6 21:30 text
-rw-r--r--  1 yehua  staff  3929042 Apr  4 16:56 text_filtered
-rw-r--r--  1 yehua  staff  1586903 Apr  6 21:30 utt2spk
-rw-r--r--  1 yehua  staff    38394 Apr  6 21:30 wav.scp


In [19]:
ls -l ../data/test

total 1960
-rw-r--r--  1 yehua  staff  193371 Apr  6 21:30 segments
-rw-r--r--  1 yehua  staff     140 Apr  6 21:30 spk2gender
-rw-r--r--  1 yehua  staff  331887 Apr  6 21:30 text
-rw-r--r--  1 yehua  staff  331887 Apr  4 16:59 text_filtered
-rw-r--r--  1 yehua  staff  123247 Apr  6 21:30 utt2spk
-rw-r--r--  1 yehua  staff    2740 Apr  6 21:30 wav.scp


a.) spk2gender 
This file informs about speakers gender. As we assumed, 'speakerID' is a unique name of each speaker.

Pattern: [speakerID] [gender]

In [21]:
def get_file_list(audio_path, dirn):
    r"""Count the number of recordings in Conversation and Interview directories, return a list of file names
    Returns
    -------
    dir_list : list
        a list of file names under the corresponding Conversation or Interview's audio directory
    """
    dir_list = os.listdir(audio_path)
    dir_list = [f for f in os.listdir(audio_path) if re.match(r'[^\\]+\.flac', f)] # makes sure unwanted files like .DS_Store doesn't show up here! 
    print ("there are total {} of files in {}\n".format(len(dir_list), audio_path))
    if dirn == 'interview':
        assert (len(dir_list) == 210), 'LDC2015S04/seame_d1/data/interview/audio should have 210 recordings, check if the directory is corrupted'
    else:
        assert (len(dir_list) == 87), 'LDC2015S04/seame_d2/data/conversation/audio should have 87 recordings, check if the directory is corrupted'
    return dir_list

def speaker_re_counts(dir_list):
    r"""Create a dictionary mapping of prefix to the number of recordings under this prefix. 
    Returns
    -------
    id_dic : collections.defaultdict
        a dictionary with key as recording prefix (tentative speaker id) and number of files associated with this prefix
        e.g. (interview) 'NI52MBQ': 2
        e.g. (conversation) '37NC45MBP': 1
    """
    id_dic = defaultdict(int)
    for file in dir_list:
        id_dic[re.split('_', file)[0]] += 1
    return id_dic

In [22]:
audio_path_i = parent_path + '/LDC2015S04/seame_d2/data/interview/audio'
audio_path_c = parent_path + '/LDC2015S04/seame_d1/data/conversation/audio'

dir_list_i = get_file_list(audio_path_i, 'interview')
dir_list_c = get_file_list(audio_path_c, 'conversation')

id_dic_i = speaker_re_counts(dir_list_i)
id_dic_c = speaker_re_counts(dir_list_c)

there are total 210 of files in /Volumes/STARTRACK/deep-learning/code-switch/speech-to-text/codeswitch/LDC2015S04/seame_d2/data/interview/audio

there are total 87 of files in /Volumes/STARTRACK/deep-learning/code-switch/speech-to-text/codeswitch/LDC2015S04/seame_d1/data/conversation/audio



In [23]:


# def count_files(audio_path):
#     dir_list = os.listdir(audio_path)[1:]
#     print ("there are total {} of files in {}\n".format(len(dir_list), audio_path))
#     return dir_list
# audio_path_i = parent_path + '/LDC2015S04/seame_d2/data/interview/audio'
# audio_path_c = parent_path + '/LDC2015S04/seame_d1/data/conversation/audio'
# dir_list_i = count_files(audio_path_i)
# dir_list_c = count_files(audio_path_c)


# def speaker_re_counts(dir_list):
#     id_dic = defaultdict(int)
#     for file in dir_list:
#         id_dic[re.split('_', file)[0]] += 1
#     print ('there are {} unique prefix sets'.format(len(id_dic)))
#     return id_dic
# id_dic_i = speaker_re_counts(dir_list_i)
# id_dic_c = speaker_re_counts(dir_list_c)

In [24]:
# test_short_ids_i = ['01MA', '03FA','08MA', '29FA','29MB','42FB','44MB','45FB','67MB','55FB']
# test_short_ids_c = ['01NC01FB', '01NC02FB','06NC11MA', '06NC12MA']

# def train_test_split(id_dic, test_short_ids):
#     train_ids = []
#     test_ids = []
#     for key in id_dic:
#         if key[2:-1] in test_short_ids or key[:-1] in test_short_ids:
#             test_ids.append(key)
#         else: 
#             train_ids.append(key)
#     print ('there are {} speaker ids in the training set'.format(len(train_ids)))
#     print ('there are {} speaker ids in the testing set\n'.format(len(test_ids)))
#     return train_ids, test_ids
# train_ids_i, test_ids_i = train_test_split(id_dic_i, test_short_ids_i)
# train_ids_c, test_ids_c = train_test_split(id_dic_c, test_short_ids_c)

In [25]:
def train_test_split(id_dic, test_short_ids, dirn):
    r"""generate a list of speaker id that should be the train or test set 
    Returns
    -------
    train_ids, test_ids : list, list
        a list of ids that belong to train or test set 
        e.g. (interview train) 'NI52MBQ' (interview test) 'NI55FBP'
        e.g. (conversation train) '37NC45MBP' (interview test) '01NC02FBY'
    """
    train_ids = []
    test_ids = []
    for key in id_dic:
        if key[2:-1] in test_short_ids or key[:-1] in test_short_ids:
            test_ids.append(key)
        else: 
            train_ids.append(key)
    print ('there are {} unprocessed speaker ids in the training set'.format(len(train_ids)))
    print ('there are {} unprocessed speaker ids in the testing set\n'.format(len(test_ids)))
    
    if dirn == "interview":
        assert (len(train_ids) == 85 and len(test_ids) == 10), "For interview, there should be 85 speakers be moved to the training set, 10 speakers in test set"
    else:
        assert (len(train_ids) == 75 and len(test_ids) == 4), "For conversation, there should be 75 speakers be moved to the training set, 4 speakers in test set"
    
    return train_ids, test_ids

In [26]:
test_short_ids_i = ['01MA', '03FA','08MA', '29FA','29MB','42FB','44MB','45FB','67MB','55FB']
test_short_ids_c = ['01NC01FB', '01NC02FB','06NC11MA', '06NC12MA']

train_ids_i, test_ids_i = train_test_split(id_dic_i, test_short_ids_i, "interview")
train_ids_c, test_ids_c = train_test_split(id_dic_c, test_short_ids_c, "conversation")

there are 85 unprocessed speaker ids in the training set
there are 10 unprocessed speaker ids in the testing set

there are 75 unprocessed speaker ids in the training set
there are 4 unprocessed speaker ids in the testing set



In [27]:
print (test_ids_i[:5])
print (train_ids_i[:5])
print (test_ids_c[:5])
print (train_ids_c[:5])

['NI44MBQ', 'NI45FBP', 'NI67MBQ', 'UI08MAZ', 'NI42FBQ']
['UI20MAZ', 'UI17FAZ', 'NI22FBP', 'NI14MBP', 'NI37MBP']
['06NC11MAX', '01NC02FBY', '01NC01FBX', '06NC12MAY']
['41NC59MAX', '22NC44MBQ', '23NC35FBQ', '12NC23FBP', '15NC29FBP']


In [28]:
print ("parent path: {}".format(parent_path))

parent path: /Volumes/STARTRACK/deep-learning/code-switch/speech-to-text/codeswitch


In [449]:
def generate_spk2gender(id_list, dirt, dirn):
    r"""create spk2gender file under data/train or data/test
    Returns
    -------
    None
    """
    if dirt == "train":
        directory = parent_path + "/data/train/spk2gender"
    elif dirt == "test":
        directory = parent_path + "/data/test/spk2gender"
    with open(directory, 'a+') as outfile:
        for speakerid in id_list:
            if dirn == "conversation":
                speakerid = speakerid[2:]
                if speakerid == "NC50XFB": # this speaker is originally gender unknown, to prevent it from being filtered out by Kaldi, I assign it to be female
                    gender = 'f'
                else:
                    gender = speakerid[4].lower()
            else:
                gender = speakerid[4].lower()
            outfile.write("{} {}\n".format(speakerid, gender))

In [450]:
def dir_clean_up(dir_path):
    files = glob.glob(dir_path + '/*')
    for f in files:
        rmtree(f)

In [451]:
def file_clean_up(dir_path, fn):
    to_be_removed = dir_path + "/" + fn
    if os.path.isfile(to_be_removed):
        os.remove(to_be_removed)

In [452]:
# clean up whatever is already in data/train or data/test
file_clean_up(parent_path + "/data/train", "spk2gender")
file_clean_up(parent_path + "/data/test", "spk2gender")

In [453]:
ls -l ../data/test

total 648
-rw-r--r--  1 yehua  staff  196988 Apr 14 17:05 segments
-rw-r--r--  1 yehua  staff  125578 Apr 14 17:05 utt2spk
-rw-r--r--  1 yehua  staff    2748 Apr 14 23:20 wav.scp


In [454]:
generate_spk2gender(train_ids_i, "train", "interview")
generate_spk2gender(train_ids_c, "train", "conversation")
generate_spk2gender(test_ids_i, "test", "interview")
generate_spk2gender(test_ids_c, "test", "conversation")
print ("finish generating spk2gender!\n")

finish generating spk2gender!



In [414]:
%%bash
wc -l ../data/test/spk2gender

      14 ../data/test/spk2gender


In [455]:
less ../data/train/spk2gender


In [456]:
%%bash
grep x ../data/train/spk2gender

In [74]:
# # add interview speaker id to the train 
# directory = parent_path + "/data/train/spk2gender"
# with open(directory, 'w') as outfile:
#     for speakerid in train_ids_i:
#         outfile.write("{} {}\n".format(speakerid,speakerid[4].lower()))

In [75]:
# # add conversation speaker id to the train 
# directory = parent_path + "/data/train/spk2gender"
# with open(directory, 'a+') as outfile:
#     for speakerid in train_ids_c:
#         outfile.write("{} {}\n".format(speakerid[2:],speakerid[6].lower()))

In [76]:
# # add interview speaker id to the test 
# directory = parent_path + "/data/test/spk2gender"
# with open(directory, 'w') as outfile:
#     for speakerid in test_ids_i:
#         outfile.write("{} {}\n".format(speakerid, speakerid[4].lower()))

In [77]:
# # add conversation speaker id to the test 
# directory = parent_path + "/data/test/spk2gender"
# with open(directory, 'a+') as outfile:
#     for speakerid in test_ids_c:
#         outfile.write("{} {}\n".format(speakerid[2:], speakerid[6].lower()))
        
# print ("finish creating spk2gender in train and test set ")

In [121]:
less ../data/test/spk2gender

In [97]:
less ../data/train/spk2gender

b.) wav.scp 
This file connects every utterance (sentence said by one person during particular recording session) with an audio file related to this utterance. If you stick to my naming approach, 'utteranceID' is nothing more than 'speakerID' (speaker's folder name) glued with *.wav file name without '.wav' ending (look for examples below).

Pattern: [recordingID] [full_path_to_audio_file]

In [81]:
ls -l ../interview_audio/test/UI03FAZ

total 35456
-rw-r--r--  1 yehua  staff  18149573 Mar 29 00:29 UI03FAZ_0101.flac


In [229]:
def generate_wavscp(dir_list, id_list, dirt, dirn):
    r"""generate wav.scp in data/train or data/test 
    the pattern for each line of the wav.scp file is <recording_id><full_path_to_audio_file> 
    because our audio is in flac, I need to tell Kaldi to convert it at some time
    e.g. NC07FBX_040201 flac -c -d -s /codeswitch/audio/train/NC07FBX/NC07FBX_040201.flac |
    Returns
    -------
    None 
    """
    if dirt == "train":
        directory = parent_path + "/data/train/wav.scp"
    else:
        directory = parent_path + "/data/test/wav.scp"
    with open(directory, 'a+') as outfile:
        counter = 0
        for file in dir_list:
            speaker_id = re.split("_", file)[0]
            if dirt == "train" and dirn == "conversation":
                if speaker_id in id_list:
                    counter += 1
                    if speaker_id[2:] in speaker_multiple: # rename e.g. 04NC07FBX_0201.flac to NC07FBX_040201.flac
                        pre = re.split("_",file)[0][:2]
                        end = re.split("_",file)[1].split(".")[0]
                        newfile = re.split("_",file)[0][2:] + '_' + pre + end + ".flac"
                        path = parent_path + "/audio/train/" + speaker_id[2:] + "/" + newfile
                        recording_id = re.split("\.", newfile)[0]
                        #print ("this {} has multiple recordings, renaming them to {}".format(speaker_id, newfile))
                    else: 
                        path = parent_path + "/audio/train/" + speaker_id[2:] + "/" + file[2:]
                        recording_id = re.split("\.", file)[0][2:]
                    
                    outfile.write("{} flac -c -d -s {} |\n".format(recording_id, path))

            else: 
                if dirn == "interview" and speaker_id in id_list and dirt =="train":
                    counter += 1
                    recording_id = re.split("\.", file)[0]
                    path = parent_path + "/audio/train/" + speaker_id + "/" + file
                    outfile.write("{} flac -c -d -s {} |\n".format(recording_id, path))
                elif dirn == "interview" and speaker_id in id_list and dirt == "test":
                    counter += 1
                    recording_id = re.split("\.", file)[0]
                    path = parent_path + "/audio/test/" + speaker_id + "/" + file
                    outfile.write("{} flac -c -d -s {} |\n".format(recording_id, path))
                elif dirn == "conversation" and speaker_id in id_list and dirt == "test":
                    counter += 1
                    recording_id = re.split("\.", file)[0][2:]
                    path = parent_path + "/audio/test/" + speaker_id[2:] + "/" + file[2:]
                    outfile.write("{} flac -c -d -s {} |\n".format(recording_id, path))
                    
        if dirt == "train" and dirn == "conversation":
            assert(counter == 83), "should write 83 lines from conversation to wav.scp (train)"
        elif dirt == "test" and dirn == "conversation":
            assert(counter == 4), "should write 4 lines from conversation to wav.scp (test)"
        elif dirt == "train" and dirn == "interview":
             assert(counter == 194), "should write 194 lines from interview to wav.scp (train)"
        else:
            assert(counter == 16), "should write 16 lines from interview to wav.scp (test)"

In [416]:
# clean up existing wav.scp 
file_clean_up(parent_path + "/data/train", "wav.scp")
file_clean_up(parent_path + "/data/test", "wav.scp")

In [417]:
speaker_multiple = ['NC50FBP', 'NC44MBQ', 'NC45MBP', 'NC05FAX', 'NC49FBQ', 'NC41MBP', 'NC07FBX', 'NC03FBX', 'NC04FBY', 'NC10MAY', 'NC37MBP', 'NC36MBQ', 'NC35FBQ', 'NC22MBQ', 'NC08FBY', 'NC48FBP', 'NC06FAY', 'NC43FBQ', 'NC09FAX']
generate_wavscp(dir_list_c, train_ids_c, "train", "conversation")
generate_wavscp(dir_list_i, train_ids_i, "train", "interview")
generate_wavscp(dir_list_c, test_ids_c, "test", "conversation")
generate_wavscp(dir_list_i, test_ids_i, "test", "interview")
print ("finish generating wav.scp!\n")

finish generating wav.scp!



In [193]:
print ("finish generating wav.scp")

finish generating wav.scp


In [232]:
less ../data/train/wav.scp

In [173]:
less ../data/test/wav.scp

In [20]:
# # add interview train into wav.scp 

# directory = parent_path + "/data/train/wav.scp"
# with open(directory, 'w') as outfile:
#     for file in dir_list_i:
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in train_ids_i:
#             path = parent_path + "/audio/train/" + speaker_id + "/" + file
#             outfile.write("{} flac -c -d -s {} |\n".format(re.split("\.", file)[0], path))
        

need to carefully re-name recordings in conversation, cause they don't have the speaker id as prefix. In their original naming convension, recordings from the same speaker will be identifies as different speaker. 

In [21]:
print(train_ids_c[:1])

['08NC15MBP']


In [22]:
# # add conversation train into wav.scp 
# speaker_multiple = ['NC50FBP', 'NC44MBQ', 'NC45MBP', 'NC05FAX', 'NC49FBQ', 'NC41MBP', 'NC07FBX', 'NC03FBX', 'NC04FBY', 'NC10MAY', 'NC37MBP', 'NC36MBQ', 'NC35FBQ', 'NC22MBQ', 'NC08FBY', 'NC48FBP', 'NC06FAY', 'NC43FBQ', 'NC09FAX']
# directory = parent_path + "/data/train/wav.scp"
# with open(directory, 'a+') as outfile:
#     for file in dir_list_c:
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in train_ids_c:
#             if speaker_id[2:] in speaker_multiple:
#                 pre = re.split("_",file)[0][:2]
#                 end = re.split("_",file)[1].split(".")[0]
#                 newfile = re.split("_",file)[0][2:] + '_' + pre + end + ".flac"
#                 path = parent_path + "/audio/train/" + speaker_id[2:] + "/" + newfile
#                 recording_id = re.split("\.", newfile)[0]
#                 #print ("this {} has multiple recordings, renaming them to {}".format(speaker_id, newfile))
#             else: 
#                 path = parent_path + "/audio/train/" + speaker_id[2:] + "/" + file[2:]
#                 recording_id = re.split("\.", file)[0][2:]
#             outfile.write("{} flac -c -d -s {} |\n".format(recording_id, path))
            

In [23]:
# # add interview test into wav.scp 

# directory = parent_path + "/data/test/wav.scp"
# with open(directory, 'w') as outfile:
#     for file in dir_list_i:
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in test_ids_i:
#             path = parent_path + "/audio/test/" + speaker_id + "/" + file
#             outfile.write("{} flac -c -d -s {} |\n".format(re.split("\.", file)[0], path))

In [174]:
# # add conversation test into wav.scp 

# directory = parent_path + "/data/test/wav.scp"
# with open(directory, 'a+') as outfile:
#     for file in dir_list_c:
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in test_ids_c:
#             path = parent_path + "/audio/test/" + speaker_id[2:] + "/" + file[2:]
#             outfile.write("{} flac -c -d -s {} |\n".format(re.split("\.", file)[0][2:], path))
# print ("finish creating wav.scp in train and test set")

c.) text 
This file contains every utterance matched with its text transcription.

Pattern: [uterranceID] [text_transcription]

In [30]:
# trans_path_i = parent_path + "/LDC2015S04/seame_d2/data/interview/transcript"
# trans_list_i = os.listdir(trans_path_i)[1:]
# trans_path_c = parent_path + "/LDC2015S04/seame_d1/data/conversation/transcript"
# trans_list_c = os.listdir(trans_path_c)[1:]
# print (trans_list_i[:2])
# print (trans_list_c[:2])

['NI01MAX_0101.txt', 'NI02FAX_0101.txt']
['01NC01FBX_0101.txt', '01NC02FBY_0101.txt']


In [234]:
ls -l ../LDC2015S04/seame_d2/data/interview/transcript_filtered/ | head -2

total 6952
-rwxr-xr-x@ 1 yehua  staff   25240 Apr 11 17:15 NI01MAX_0101.txt*


In [235]:
trans_path_i = parent_path + "/LDC2015S04/seame_d2/data/interview/transcript_filtered"
trans_list_i = [f for f in os.listdir(trans_path_i) if f.endswith(".txt")]
trans_path_c = parent_path + "/LDC2015S04/seame_d1/data/conversation/transcript_filtered"
trans_list_c = [f for f in os.listdir(trans_path_c) if f.endswith(".txt")]
assert (len(trans_list_i) == 210), "there should be 210 files in interview/transcript_filtered, check if the directory is corrupted"
assert (len(trans_list_c) == 87), "there should be 87 files in conversation/transcript_filtered, check if the directory is corrupted"
print (trans_list_i[:2])
print (trans_list_c[:2])

['NI01MAX_0101.txt', 'NI02FAX_0101.txt']
['01NC01FBX_0101.txt', '01NC02FBY_0101.txt']


In [429]:
def write_text(file_list, trans_path, id_list, dirt, dirn):
    r"""generate text file
    for each line the pattern is: <utterance_id><transcript>
    e.g. NC03FBX_0101_0008400_0010710 就 那种 TYPICAL 的 偶 像 剧
    """
    if dirt == "test":
        directory = parent_path + "/data/test/text"
    elif dirt == "train":
        directory = parent_path + "/data/train/text"
    counter = 0
    with open(directory, 'a+') as outputfile:
        for file in file_list:
            speaker_id = re.split("_", file)[0]
            if speaker_id in id_list: 
                trans_file = trans_path + "/" + file
                with open(trans_file, 'r') as inputfile: # only read file from transcript_filtered if the speaker is a match
                    for line in inputfile:
                        if dirn =="conversation" and dirt == "train":
                            if speaker_id[2:] in speaker_multiple:
                                pre = re.split("_",file)[0][:2] # e.g. 04
                                end = re.split("_",file)[1].split(".")[0] # e.g. 0201
                                prefix = re.split("_",file)[0][2:] + '_' + pre + end  #make 04NC08FBY_0201 -> NC08FBY_040201
                            else:
                                prefix = re.split(" ", line)[0][2:] #make prefix 02NC03FBX_0101 -> NC03FBX_0101
                            
                        elif dirn =="conversation" and dirt == "test":
                            prefix = re.split(" ", line)[0][2:]
                        else:
                            prefix = re.split(" ", line)[0]
                        text = " ".join(re.split(" ", line)[3:])
                        first_frame = re.split(" ", line)[1].zfill(7)
                        second_frame = re.split(" ", line)[2].zfill(7)
                        utterance_id = prefix + "_" + first_frame + "_" + second_frame
                        outputfile.write("{} {}".format(utterance_id, text))
                        counter += 1
        print ("write {} lines of {} text from {}".format(counter, dirt, dirn))
        if dirt == "train" and dirn == "conversation":
            pass
            #assert(counter == 13731), "should write 13731 lines from conversation to text (train)"
        elif dirt == "test" and dirn == "conversation":
            pass
            #assert(counter == 485, "should write 485 lines from conversation to text(test)"
        elif dirt == "train" and dirn == "interview":
            pass
            #assert(counter == 29505), "should write 29505 lines from interview to text (train)"
        else:
            pass
            #assert(counter == 2909), "should write 2909 lines from interview to wav.scp (test)" 

In [430]:
# clean up existing wav.scp 
file_clean_up(parent_path + "/data/train", "text")
file_clean_up(parent_path + "/data/test", "text")

In [433]:
write_text(trans_list_c, trans_path_c, train_ids_c, "train", "conversation")
# write_text(trans_list_i, trans_path_i, train_ids_i, "train", "interview")
# write_text(trans_list_c, trans_path_c, test_ids_c, "test", "conversation")
# write_text(trans_list_i, trans_path_i, test_ids_i, "test", "interview")
print ("finish creating text in data/train and data/test!\n")

write 13731 lines of train text from conversation
finish creating text in data/train and data/test!



In [435]:
less ../data/train/text

In [275]:
%%bash
wc -l ../data/train/text

   43236 ../data/train/text


In [41]:
# # create conversation text in train set 
# speaker_multiple = ['NC50FBP', 'NC44MBQ', 'NC45MBP', 'NC05FAX', 'NC49FBQ', 'NC41MBP', 'NC07FBX', 'NC03FBX', 'NC04FBY', 'NC10MAY', 'NC37MBP', 'NC36MBQ', 'NC35FBQ', 'NC22MBQ', 'NC08FBY', 'NC48FBP', 'NC06FAY', 'NC43FBQ', 'NC09FAX']
# directory = parent_path + "/data/train/text"
# counter = 0 
# with open(directory, 'a+') as outputfile:
#     for file in trans_list_c: 
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in train_ids_c:
#             trans_file = trans_path_c + "/" + file
#             with open(trans_file, 'r') as inputfile:
#                 for line in inputfile:
#                     if speaker_id[2:] in speaker_multiple:
#                         pre = re.split("_",file)[0][:2]
#                         end = re.split("_",file)[1].split(".")[0]
#                         prefix = re.split("_",file)[0][2:] + '_' + pre + end
#                         #print ("this speaker {} has multiple recordings, renaming it to {}".format(speaker_id, prefix))

#                     else: 
#                         prefix = re.split(" ", line)[0][2:]
#                     text = " ".join(re.split(" ", line)[3:])
#                     first_frame = re.split(" ", line)[1].zfill(7)
#                     second_frame = re.split(" ", line)[2].zfill(7)
#                     utterance_id = prefix + "_" + first_frame + "_" + second_frame
#                     outputfile.write("{} {}".format(utterance_id, text))



In [47]:
# # create conversation text in test 

# directory = parent_path + "/data/test/text"
# with open(directory, 'a+') as outputfile:
#     for file in trans_list_c: 
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in test_ids_c:
#             trans_file = trans_path_c + "/" + file
#             with open(trans_file, 'r') as inputfile:
#                 for line in inputfile:
#                     text = " ".join(re.split(" ", line)[3:])
#                     prefix = re.split(" ", line)[0][2:]
#                     first_frame = re.split(" ", line)[1].zfill(7)
#                     second_frame = re.split(" ", line)[2].zfill(7)
#                     utterance_id = prefix + "_" + first_frame + "_" + second_frame
#                     outputfile.write("{} {}".format(utterance_id, text))
                    
# print ("finish create text in train and test set")

finish create text in train and test set


In [46]:
# directory = parent_path + "/data/test/text"
# with open(directory, 'w') as outputfile:
#     for file in trans_list_i: 
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in test_ids_i:
#             trans_file = trans_path_i + "/" + file
#             with open(trans_file, 'r') as inputfile:
#                 for line in inputfile:
#                     text = " ".join(re.split(" ", line)[3:])
#                     prefix = re.split(" ", line)[0]
#                     first_frame = re.split(" ", line)[1].zfill(7)
#                     second_frame = re.split(" ", line)[2].zfill(7)
#                     utterance_id = prefix + "_" + first_frame + "_" + second_frame
#                     outputfile.write("{} {}".format(utterance_id, text))

In [40]:
# directory = parent_path + "/data/train/text"
# with open(directory, 'w') as outputfile:
#     for file in trans_list_i: 
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in train_ids_i:
#             trans_file = trans_path_i + "/" + file
#             with open(trans_file, 'r') as inputfile:
#                 for line in inputfile:
#                     text = " ".join(re.split(" ", line)[3:])
#                     #text = re.split(" ", line)[-1]
#                     prefix = re.split(" ", line)[0]
#                     first_frame = re.split(" ", line)[1].zfill(7)
#                     second_frame = re.split(" ", line)[2].zfill(7)
#                     utterance_id = prefix + "_" + first_frame + "_" + second_frame
                    outputfile.write("{} {}".format(utterance_id, text))

In [44]:
less ../data/train/text

In [35]:
test_ids_c

['06NC11MAX', '01NC02FBY', '01NC01FBX', '06NC12MAY']

In [38]:
less ../data/test/text_filtered

d.) utt2spk 
This file tells the ASR system which utterance belongs to particular speaker.

Pattern: [uterranceID] [speakerID]

In [285]:
def get_largest_frame(trans_list, trans_path):
    largest_frame = -sys.maxsize
    for file in trans_list:
        file_path = trans_path + "/" + file
        with open(file_path, 'r') as inputfile:
            for line in inputfile:
                second_frame = int(re.split(" ", line)[2])
                if second_frame > largest_frame:
                    largest_frame = second_frame
    return largest_frame

def get_frame_size(frame1, frame2):
    gframe = frame1
    if gframe < frame2:
        gframe = frame2
    size = len(list(str(gframe)))
    print ("since {} is our largest frame, then we need to create string with {} digits to hold all frames".format(gframe, size))
    return size
    
        

In [286]:
frame1 = get_largest_frame(trans_list_c, trans_path_c)
frame2 = get_largest_frame(trans_list_i, trans_path_i)
size = get_frame_size(frame1, frame2)

since 7242490 is our largest frame, then we need to create string with 7 digits to hold all frames


In [51]:
# #trans_path = parent_path + "/LDC2015S04/seame_d2/data/interview/transcript/"
# import sys
# largest_frame = -sys.maxsize
# trans_list_i = os.listdir(trans_path_i)[1:]
# print ("there are {} recording with transcript".format(len(trans_list_i)))
# print ("there are {} recording in audio files".format(len(dir_list_i)))
# print ("match!")

# for file in trans_list_i:
#     file_path = trans_path_i + "/" + file
#     with open(file_path, 'r') as inputfile:
#         for line in inputfile:
#             second_frame = int(re.split(" ", line)[2])
#             if second_frame > largest_frame:
#                 largest_frame = second_frame
# print ("largest_frame is {}".format(largest_frame))  



there are 210 recording with transcript
there are 210 recording in audio files
match!
largest_frame is 7004497


In [52]:
# for file in trans_list_c:
#     file_path = trans_path_c + "/" + file
#     with open(file_path, 'r') as inputfile:
#         for line in inputfile:
#             second_frame = int(re.split(" ", line)[2])
#             if second_frame > largest_frame:
#                 largest_frame = second_frame
# print ("largest_frame is {}".format(largest_frame))  
# print ("since 7242490 is our largest frame, then we need to create string with 7 digits to hold all frames")

largest_frame is 7242490
since 7242490 is our largest frame, then we need to create string with 7 digits to hold all frames


In [298]:
def gen_utter_list(trans_list, trans_path, dirn):
    r"""create a list of utterance_id
    e.g. ['UI29FAZ_0104_0985128_0986293']
    Returns
    -------
    utter_ids : list
        A list of utterance_id
    """
    utter_ids = []
    for file in trans_list:
        file_path = trans_path + "/" + file
        with open(file_path, 'r') as inputfile:
            for line in inputfile:
                speaker_id = re.split("_", line)[0]
                if dirn == "conversation":
                    if speaker_id[2:] in speaker_multiple:
                        pre = re.split("_",file)[0][:2]
                        end = re.split("_",file)[1].split(".")[0]
                        prefix = re.split("_",file)[0][2:] + '_' + pre + end
                    else:
                        prefix = re.split(" ", line)[0][2:]
                else:
                    prefix = re.split(" ", line)[0]
                first_frame = re.split(" ", line)[1].zfill(7)
                second_frame = re.split(" ", line)[2].zfill(7)
                utterance_id =  prefix + "_" + first_frame + "_" + second_frame
                utter_ids.append(utterance_id)
    print ("{} has {} utterance ids in total".format(dirn, len(utter_ids)))
    return utter_ids

In [303]:
utter_ids_i = gen_utter_list(trans_list_i, trans_path_i, "interview")
utter_ids_c = gen_utter_list(trans_list_c, trans_path_c, "conversation")

interview has 32414 utterance ids in total
conversation has 14216 utterance ids in total


In [53]:
# # create utterance id for interview: recording id + start time + end time; for e.g. NI01MAX_0101_0001353_0003612
# counter = 0
# utter_ids_i = []
# for file in trans_list_i:
#     file_path = trans_path_i + "/" + file
#     with open(file_path, 'r') as inputfile:
#         for line in inputfile:
#             speaker_id = re.split("_", line)[0]
#             prefix = re.split(" ", line)[0]
#             first_frame = re.split(" ", line)[1].zfill(7)
#             second_frame = re.split(" ", line)[2].zfill(7)
#             utterance_id =  prefix + "_" + first_frame + "_" + second_frame
#             utter_ids_i.append(utterance_id)
# print ("there are {} new utterance ids".format(len(utter_ids_i)))


there are 31658 new utterance ids


In [55]:
utter_ids_i[-1]

'UI29FAZ_0104_0985128_0986293'

In [56]:
# # create utterance id for conversation

# counter = 0
# utter_ids_c = []
# for file in trans_list_c:
#     file_path = trans_path_c + "/" + file
#     with open(file_path, 'r') as inputfile:
#         for line in inputfile:
#             speaker_id = re.split("_", line)[0]
#             if speaker_id[2:] in speaker_multiple:
#                 pre = re.split("_",file)[0][:2]
#                 end = re.split("_",file)[1].split(".")[0]
#                 prefix = re.split("_",file)[0][2:] + '_' + pre + end
#             else:
#                 prefix = re.split(" ", line)[0][2:]
#             first_frame = re.split(" ", line)[1].zfill(7)
#             second_frame = re.split(" ", line)[2].zfill(7)
#             utterance_id =  prefix + "_" + first_frame + "_" + second_frame
#             utter_ids_c.append(utterance_id)
# print ("there are {} new utterance ids".format(len(utter_ids_c)))

there are 14214 new utterance ids


In [307]:
utter_ids_c[-1]

'NC41MBP_460101_3631302_3632862'

In [304]:
print("sample newly created utterance id {}".format(utter_ids_i[:1]))

sample newly created utterance id ['NI01MAX_0101_0001353_0003612']


In [305]:
print("sample newly created utterance id {}".format(utter_ids_c[:1]))

sample newly created utterance id ['NC01FBX_0101_0086300_0088370']


In [306]:
train_ids_c_short = [ x[2:] for x in train_ids_c]

In [62]:
print (train_ids_c_short[:2])

['NC15MBP', 'NC42MBQ']


In [330]:
def create_utt2spk(id_list, utter_ids, dirt, dirn):
    r"""create utt2spk file in data/train or data/test
    For each line of utt2spk file the pattern is <utterance_id><speaker_id>
    e.g. NC01FBX_0101_0086300_0088370 NC01FBX
    Returns
    -------
    None
    """
    counter = 0
    if dirt == "train":
        utt2spk_path = parent_path + "/data/train/utt2spk"
    else:
        utt2spk_path = parent_path + "/data/test/utt2spk"
    if dirn == "conversation":
        id_list = [x[2:] for x in id_list]
    with open(utt2spk_path, 'a+') as outputfile:
        for file in utter_ids:
            speaker_id = re.split("_", file)[0]
            if speaker_id in id_list:
                counter += 1
                outputfile.write("{} {}\n".format(file, speaker_id))
    print ("write {} lines of utt2spk from {}".format(counter, dirn))

In [334]:
# clean up any exisiting utt2spk
file_clean_up(parent_path+"/data/train", 'utt2spk')
file_clean_up(parent_path+"/data/test", 'utt2spk')

In [335]:
create_utt2spk(train_ids_c, utter_ids_c, "train", "conversation")
create_utt2spk(train_ids_i, utter_ids_i, "train", "interview")
create_utt2spk(test_ids_c, utter_ids_c, "test", "conversation")
create_utt2spk(test_ids_i, utter_ids_i, "test", "interview")

write 13731 lines of utt2spk from conversation
write 29505 lines of utt2spk from interview
write 485 lines of utt2spk from conversation
write 2909 lines of utt2spk from interview


In [316]:
utter_ids_c[:2]

['NC01FBX_0101_0086300_0088370', 'NC01FBX_0101_0165090_0167860']

In [317]:
train_ids_c[:2]

['41NC59MAX', '22NC44MBQ']

In [65]:
# test_ids_c_short = [x[2:] for x in test_ids_c]
# utt2spk_path = parent_path + "/data/test/utt2spk"
# counter = 0
# with open(utt2spk_path, 'w') as outputfile:
#     for file in utter_ids_i:
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in test_ids_i:
#             counter += 1
#             outputfile.write("{} {}\n".format(file, speaker_id))
#     for file in utter_ids_c:
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in test_ids_c_short:
#             counter += 1
#             outputfile.write("{} {}\n".format(file, speaker_id))
# print ("there are {} in test/utt2spk".format(counter))


there are 3331 in test/utt2spk


In [63]:
# utt2spk_path = parent_path + "/data/train/utt2spk"
# counter = 0
# with open(utt2spk_path, 'w') as outputfile:
#     for file in utter_ids_i:
#         speaker_id = re.split("_", file)[0]
#         if speaker_id in train_ids_i:
#             counter += 1
#             outputfile.write("{} {}\n".format(file, speaker_id))
#     for file in utter_ids_c:
#         speaker_id = re.split("_",file)[0]
#         if speaker_id in train_ids_c_short:
#             counter += 1
#             outputfile.write("{} {}\n".format(file, speaker_id))
# print ("there are {} in train/utt2spk".format(counter))

there are 42541 in train/utt2spk


In [64]:
test_ids_c

['06NC11MAX', '01NC02FBY', '01NC01FBX', '06NC12MAY']

In [338]:
less ../data/train/utt2spk

In [339]:
less ../data/test/utt2spk

e.) corpus.txt 
This file has a slightly different directory. In kaldi-trunk/egs/digits/data create another folder local. In kaldi/egs/code-switching/data/local create a file corpus.txt which should contain every single utterance transcription that can occur in your ASR system (in our case it will be 100 lines from 100 audio files).

Pattern: [text_transcription]

In [340]:
trans_path_c


'/Volumes/STARTRACK/deep-learning/code-switch/speech-to-text/codeswitch/LDC2015S04/seame_d1/data/conversation/transcript_filtered'

In [342]:
len(trans_list_i)

210

In [343]:
trans_list_i[:3]

['NI01MAX_0101.txt', 'NI02FAX_0101.txt', 'NI03FBX_0101.txt']

In [347]:
trans_list_c[:3]

['01NC01FBX_0101.txt', '01NC02FBY_0101.txt', '02NC03FBX_0101.txt']

In [345]:
train_ids_c[:2]

['41NC59MAX', '22NC44MBQ']

In [346]:
train_ids_i[:2]

['UI20MAZ', 'UI17FAZ']

In [380]:
def write_corpus(trans_list, trans_path, train_ids, dirn):
    r"""write only transcript used in training to corpus.txt
    each line of the corpus.txt has pattern <text_transcription>
    e.g. 就是 那种 TYPICAL 偶 像 剧
    Returns
    -------
    None
    """
    counter = 0
    temp_path = parent_path + "/data/local"
    if not os.path.exists(temp_path):
        os.makedirs(temp_path) 
    corpus_path = parent_path + "/data/local/corpus.txt"
    with open(corpus_path, 'a+') as outputfile:
        for file in trans_list:
            speaker_id = re.split("_",file)[0]
            if speaker_id in train_ids: # only take in training set transcript to over information leakage
                trans_file = trans_path + "/" + file
                with open(trans_file, 'r') as inputfile:
                    for line in inputfile:
                        counter += 1
                        outputfile.write(" ".join(re.split(" ", line)[3:]))
    print ("{} lines of transcript from {} is written to data/local/corpus.txt (training data only)".format(counter, dirn))

In [381]:
# clean up any exisiting utt2spk
file_clean_up(parent_path+"/data/local", 'corpus.txt')
file_clean_up(parent_path+"/data/local", 'corpus.txt')


In [382]:
ls ../data/local

lang/


In [383]:
write_corpus(trans_list_c, trans_path_c, train_ids_c, "conversation")
write_corpus(trans_list_i, trans_path_i, train_ids_i, "interview")
print("finish creating data/lang/corpus.txt!\n")

13731 lines of transcript from conversation is written to data/local/corpus.txt (training data only)
29505 lines of transcript from interview is written to data/local/corpus.txt (training data only)
finish creating data/lang/corpus.txt!



In [68]:
# temp_path = parent_path + "/data/local"
# if not os.path.exists(temp_path):
#     os.makedirs(temp_path)
    
# corpus_path = parent_path + "/data/local/corpus.txt"
# trans_path = parent_path + "/LDC2015S04/seame_d2/data/interview/transcript_filtered"
# trans_list = os.listdir(trans_path)[1:]

# with open(corpus_path, 'w') as outputfile:
#     for file in trans_list: 
#             trans_file = trans_path + "/" + file
#             with open(trans_file, 'r') as inputfile:
#                 for line in inputfile:
#                     #outputfile.write(line)
#                     outputfile.write(" ".join(re.split(" ", line)[3:]))


In [69]:
# temp_path = parent_path + "/data/local"
# if not os.path.exists(temp_path):
#     os.makedirs(temp_path)
    
# corpus_path = parent_path + "/data/local/corpus.txt"
# trans_path = parent_path + "/LDC2015S04/seame_d1/data/conversation/transcript_filtered"
# trans_list = os.listdir(trans_path)[1:]

# with open(corpus_path, 'a+') as outputfile:
#     for file in trans_list: 
#             trans_file = trans_path + "/" + file
#             with open(trans_file, 'r') as inputfile:
#                 for line in inputfile:
#                     #outputfile.write(line)
#                     outputfile.write(" ".join(re.split(" ", line)[3:]))


In [71]:
len(trans_list)

87

In [374]:
less ../data/local/corpus.txt

(f) segments file  
the format of the "segments" file is:  
[utterance-id] [recoding-id] [segment-begin] [segment-end]   
string.split("\_", 2) # split up to the second of occurences of _


In [395]:
def gen_segments(utter_ids, id_list, dirt, dirn):
    r"""create segments file under data/train or data/test
    for each line of this file has pattern <utterance_id><recording_id><segments_begin><segments_end>,
    where <segments_begin> and <segments_end> are in second
    e.g. NC01FBX_0101_0086300_0088370 NC01FBX_0101 86.3 88.37
    Returns
    -------
    None
    """
    counter = 0
    if dirt == "train":
        directory = parent_path + "/data/train/segments"
    else:
        directory = parent_path + "/data/test/segments"
    if dirn == "conversation":
        id_list = [i[2:] for i in id_list]
    with open(directory, 'a+') as outputfile:
        for utt in utter_ids:
            speaker_id = re.split("_", utt)[0]
            if speaker_id in id_list:
                recording_id = "_".join(utt.split("_", 2)[:2])
                segment_begin = str(int(re.split("_", utt)[2])/1000.0)
                segment_end = str(int(re.split("_", utt)[3])/1000.0)
                counter += 1
                outputfile.write("{} {} {} {}\n".format(utt, recording_id, segment_begin, segment_end))
    print ("{} of lines of {} is written to segments in data/{}".format(counter, dirn, dirt))

In [396]:
utter_ids_c[:3]

['NC01FBX_0101_0086300_0088370',
 'NC01FBX_0101_0165090_0167860',
 'NC01FBX_0101_0275720_0281420']

In [397]:
# clean up any exisiting utt2spk
file_clean_up(parent_path+"/data/train", 'segments')
file_clean_up(parent_path+"/data/test", 'segments')

In [398]:
ls ../data/train

spk2gender  text        utt2spk     wav.scp


In [399]:
gen_segments(utter_ids_c, train_ids_c, "train", "conversation")
gen_segments(utter_ids_i, train_ids_i, "train", "interview")
gen_segments(utter_ids_c, test_ids_c, "test", "conversation")
gen_segments(utter_ids_i, test_ids_i, "test", "interview")
print ("finish creating segments file!\n")

13731 of lines of conversation is written to segments in data/train
29505 of lines of interview is written to segments in data/train
485 of lines of conversation is written to segments in data/test
2909 of lines of interview is written to segments in data/test


In [72]:

# # interview segments file for training set
# directory = parent_path + "/data/train/segments"
# counter = 0
# with open(directory, 'w') as outputfile:
#     for utt in utter_ids_i:
#         speaker_id = re.split("_", utt)[0]
#         if speaker_id in train_ids_i:
            
#             recording_id = "_".join(utt.split("_", 2)[:2])
#             segment_begin = str(int(re.split("_", utt)[2])/1000)
#             segment_end = str(int(re.split("_", utt)[3])/1000)
#             outputfile.write("{} {} {} {}\n".format(utt, recording_id, segment_begin, segment_end))
    
        




In [73]:
utter_ids_c[:2]

['NC01FBX_0101_0086300_0088370', 'NC01FBX_0101_0165090_0167860']

In [74]:
# # conversation segments file for training set

# directory = parent_path + "/data/train/segments"
# counter = 0
# with open(directory, 'a+') as outputfile:
#     for utt in utter_ids_c:
#         speaker_id = re.split("_", utt)[0]
#         if speaker_id in train_ids_c_short:
            
#             recording_id = "_".join(utt.split("_", 2)[:2])
#             segment_begin = str(int(re.split("_", utt)[2])/1000)
#             segment_end = str(int(re.split("_", utt)[3])/1000)
#             outputfile.write("{} {} {} {}\n".format(utt, recording_id, segment_begin, segment_end))
    

In [75]:
# # interview segments file for training set
# directory = parent_path + "/data/test/segments"
# counter = 0 
# with open(directory, 'w') as outputfile:
#     for utt in utter_ids_i:
#         speaker_id = re.split("_", utt)[0]
#         if speaker_id in test_ids_i:
# #             if speaker_id == 'NI01MAX':
# #                 #print (speaker_id)
#             recording_id = "_".join(utt.split("_", 2)[:2])
#             segment_begin = str(int(re.split("_", utt)[2])/1000)
#             segment_end = str(int(re.split("_", utt)[3])/1000)
#             outputfile.write("{} {} {} {}\n".format(utt, recording_id, segment_begin, segment_end))
#             counter += 1
# print (counter)

2846


In [76]:
# # conversation segments file for test set
# directory = parent_path + "/data/test/segments"
# counter = 0 
# with open(directory, 'a+') as outputfile:
#     for utt in utter_ids_c:
#         speaker_id = re.split("_", utt)[0]
#         if speaker_id in test_ids_c_short:
# #             if speaker_id == 'NI01MAX':
# #                 #print (speaker_id)
#             recording_id = "_".join(utt.split("_", 2)[:2])
#             segment_begin = str(int(re.split("_", utt)[2])/1000)
#             segment_end = str(int(re.split("_", utt)[3])/1000)
#             outputfile.write("{} {} {} {}\n".format(utt, recording_id, segment_begin, segment_end))
#             counter += 1
# print (counter)

485


In [400]:
less ../data/train/segments

In [457]:
less ../data/test/segments

In [402]:
utter_ids_c[:2]

['NC01FBX_0101_0086300_0088370', 'NC01FBX_0101_0165090_0167860']

In [1]:
print ("\n (☞ﾟ∀ﾟ)☞ acoustic data prep is succesfully finished!" )


 (☞ﾟ∀ﾟ)☞ acoustic data prep is succesfully finished!
